In [1]:
import numpy as np


In [2]:
import requests

url_test = 'https://oda.ft.dk/api/Afstemning?$expand=Sagstrin/Sag&$filter=vedtaget eq true&$inlinecount=allpages'

r = requests.get(url_test)

sager = r.json()

In [3]:
#This function takes dict, extracts the next link, gets the new dict and appends it to the old dict

def get_all_pages(url):

    r = requests.get(url)

    sager = r.json()
    prev = r.json()

    while 'odata.nextLink' in prev:

        r = requests.get(prev['odata.nextLink'])
        sager['value'] += r.json()['value']
        prev = r.json()

    return sager

In [4]:
sager = get_all_pages(url_test)
sager.keys()

dict_keys(['odata.metadata', 'odata.count', 'value', 'odata.nextLink'])

In [5]:
#checking if all sager have the same keys
test = [len(sager['value'][i].keys()) for i in range(len(sager['value']))]
np.unique(test)

array([ 9, 10])

In [6]:
#they don't, so we check how many sager have missing keys
missing_keys = [i for i in range(len(sager['value'])) if len(sager['value'][i].keys()) != 10]
missing_keys

[0,
 1099,
 1100,
 1327,
 1404,
 1531,
 1532,
 1533,
 1542,
 1544,
 1550,
 1556,
 1559,
 1560]

In [7]:
#the above is only 14, so we remove those sager
clean_sager = [sag for sag in sager['value'] if len(sag) == 10]

In [8]:
clean_sager[0].keys()

dict_keys(['Sagstrin', 'id', 'nummer', 'konklusion', 'vedtaget', 'kommentar', 'mødeid', 'typeid', 'sagstrinid', 'opdateringsdato'])

In [ ]:
#export clean sag to json
import json
with open('clean_sager.json', 'w') as f:
    json.dump(clean_sager, f)